## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Andrés Castillo Nassar 1168530


In [16]:
# Incluye aquí todas las librerías y paquetes que requieras.

import pandas as pd
import numpy as np
from numpy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from scipy.stats import pearsonr




### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga el archivo "restaurant+consumer+data.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [2]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("/content/sample_data/rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("/content/sample_data/geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


**NOTA: Recuerda que cada vez que se te pida incluir líneas de código, podrás incluir 1 o más líneas, pero siempre en la sección donde se te indica hacerlo.**

## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++


Se utiliza principalmente para especificar la codificación de caracteres especiales, acentos y otros símbolos que son comunes en idiomas europeos, incluyendo el español. Especificar 'latin-1' asegura que se lean y procesen correctamente.



++++++++ Termina la sección de agregar texto. +++++++++++

In [3]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [4]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [5]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [6]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1a, data2a, on='placeID')



# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [7]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(index='name', columns='userID', values='food_rating')



# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [8]:
# a. Aplica la transformación SVD para obtener la matriz de las variables
#    latentes de los restaurantes, en relación a la manera en que los usuarios
#    evaluaron la comida. Deberás obtener la factorización SVD con el máximo número
#    de componentes y por lo tanto el máximo de valores singulares posibles de acuerdo
#    a este criterio. Así, de acuerdo al resultado obtenido en el Ejercicio-3, asigna
#    a la variable "nc_food" el número entero adecuado para llegar a obtener el
#    resultado pedido.


# ************* Inlcuye aquí tu código:**********************************************

UtMx_food_filled = UtMx_food.fillna(0).values
U, Sigma, VT = svd(UtMx_food_filled, full_matrices=False)


nc_food = min(U.shape[0], VT.shape[0])    # número de componentes

U_food = U[:, :nc_food]
Sigma_food = np.diag(Sigma[:nc_food])
VT_food = VT[:nc_food, :]

# *********** Aquí termina la sección de agregar código *****************************

print('Total de valores singulares basados en evaluación de la comida:', nc_food)

Total de valores singulares basados en evaluación de la comida: 129


In [9]:
# b. Lleva a cabo la factorización SVD truncada en la matriz de utilidad
#    obtenida previamente y con respecto al número de componentes dada
#    por la variable nc_food.


# ************* Inlcuye aquí tu código:********************************


SVD_food = TruncatedSVD(n_components=nc_food)
UtMx_food_truncated = SVD_food.fit_transform(UtMx_food_filled)


# *********** Aquí termina la sección de agregar código ***************


# Con la factorización SVD truncada obtenida, determinemos la cantidad
# de componentes que explican un 90% de la variabilidad acumulada de
# cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1


print('Valor de truncamiento al 90% de dicha variabilidad:', N_food)

Valor de truncamiento al 90% de dicha variabilidad: 51


In [12]:
# Usando la métrica de correlación de Pearson, obtenemos ahora las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similitudes con el "Restaurante Pueblo Bonito"
# y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtenemos las 10 mejores correlaciones positivas a continuación.

UtMx_food_filled = UtMx_food.fillna(0)
# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food_filled)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:5]

Total de similaridades positivas encontradas: 70
Algunos de los resultados encontrados:


[(0.021873086944774436, 'Cabana Huasteca'),
 (0.029705536699362198, 'Cafe Chaires'),
 (0.014409452146524646, 'Cafeteria cenidet'),
 (0.22261070720884765, 'Cafeteria y Restaurant El Pacifico'),
 (0.011750136166660922, 'Cenaduria El RincÃ³n de Tlaquepaque')]

## **Ejercicio - 5**

In [13]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente. Llamaremos "mejores_sim_food_ordenadas" a
# dicha lista.

# ************* Inlcuye aquí tu código:*****************************



mejores_sim_food_ordenadas = sorted(mejores_sim_food, key=lambda x: x[0], reverse=True)




# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.7937715545579649, 'Restaurante la Estrella de Dima')
2> (0.6974994163218559, 'pizza clasica')
3> (0.6637620969750639, 'Restaurante Guerra')
4> (0.5446397342526144, 'El Club')
5> (0.3675588772382414, 'Pizzeria Julios')
6> (0.31189516370271436, 'Restaurant Oriental Express')
7> (0.3083490191782438, 'Restaurante El Cielo Potosino')
8> (0.30480299721667325, 'Hamburguesas saul')
9> (0.28419559957519575, 'Rincon Huasteco')
10> (0.26909639671062735, 'la Cochinita Pibil Restaurante Yucateco')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación de un restaurante con respecto a su servicio (service_rating), encuentra ahora los diez restaurantes con mayor similaridad al mismo restaurante anterior llamado "Restaurante Pueblo Bonito" y con respecto también a la misma métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [15]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************


UtMx_service = df_combinado.pivot_table(index='name', columns='userID', values='service_rating')


# *********** Aquí termina la sección de agregar código *************




print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head(10)

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cafeteria y Restaurant El Pacifico,NaN,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN
Carls Jr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carnitas Mata Calle 16 de Septiembre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carreton de Flautas y Migadas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************
UtMx_service_filled = UtMx_service.fillna(0)

svd = TruncatedSVD(n_components=UtMx_service_filled.shape[1]-1)
svd.fit(UtMx_service_filled)
explained_variance = np.cumsum(svd.explained_variance_ratio_)
n_components_90 = np.argmax(explained_variance >= 0.90) + 1

svd_90 = TruncatedSVD(n_components=n_components_90)
UtMx_service_truncated = svd_90.fit_transform(UtMx_service_filled)

pueblo_bonito_index = UtMx_service.index.get_loc('Restaurante Pueblo Bonito')

similarities = []
for i in range(UtMx_service_truncated.shape[0]):
    if i != pueblo_bonito_index:
        similarity = pearsonr(UtMx_service_truncated[pueblo_bonito_index], UtMx_service_truncated[i])[0]
        similarities.append((similarity, UtMx_service.index[i]))

similarities_sorted = sorted(similarities, key=lambda x: x[0], reverse=True)

top_10_similar_restaurants = similarities_sorted[:10]

for similarity, restaurant in top_10_similar_restaurants:
    print(f"Restaurante: {restaurant}, Similaridad: {similarity:.4f}")



# *********** Aquí termina la sección de agregar código *************

Restaurante: pizza clasica, Similaridad: 0.6669
Restaurante: El Club, Similaridad: 0.5855
Restaurante: Hamburguesas saul, Similaridad: 0.4960
Restaurante: Restaurante la Estrella de Dima, Similaridad: 0.4699
Restaurante: Restaurante Guerra, Similaridad: 0.4302
Restaurante: Restaurant Orizatlan, Similaridad: 0.4003
Restaurante: Restaurante El Cielo Potosino, Similaridad: 0.3831
Restaurante: la Cochinita Pibil Restaurante Yucateco, Similaridad: 0.3550
Restaurante: La Fontana Pizza Restaurante and Cafe, Similaridad: 0.3347
Restaurante: Restaurant Oriental Express, Similaridad: 0.3268


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

++++++++ Inicia la sección de agregar texto:

Creé dos matrices de utilidad (Utmx_food & Utmx_service). Estas matrices tienen en las filas los nombres de los restaurantes, y en las columnas los usuarios. La factorización SVD nos funcionó para reducir la dimensionalidad de los datos y capturar las características latentes más relevantes.

Al calcular la similaridad de Pearson entre el restaurante Pueblo Bonito y los demás, tomamos las calificaciones de servicio. Al ordenar los datos identificamos los diez restaurantes con mayor similaridad de Pearson. "pizza clasica" tuvo la mayor similaridad con Pueblo Bonito, seguido de El Club y Hamburguesas saul.

+++++++++++


++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**